In [1]:
import os

In [2]:
print('h')

h


In [3]:
%pwd

'd:\\ChestCancer\\MLOps\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\ChestCancer\\MLOps'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/yashviiishah/MLOps.mlflow/"
os.environ["MLFLOW_TRACKING_USERNAME"]="yashviishah"
os.environ["MLFLOW_TRACKING_PASSWORD"]="3231a18250cb655d859be9f158ddaf2e3716ab82"


In [7]:
import mlflow

mlflow.set_tracking_uri("https://dagshub.com/yashviiishah/MLOps.mlflow/")


In [8]:
import tensorflow as tf

In [9]:

model = tf.keras.models.load_model("artifacts/training/model.h5")

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [11]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [12]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Data",
            mlflow_uri="https://dagshub.com/yashviiishah/MLOps.mlflow/",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [13]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [14]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [15]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-04-15 03:13:34,116: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-15 03:13:34,120: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-15 03:13:34,122: INFO: common: created directory at: artifacts]
Found 144 images belonging to 2 classes.
9/9 [==============================] - 8s 843ms/step - loss: 0.4649 - accuracy: 0.8819
[2025-04-15 03:13:42,849: INFO: common: json file saved at: scores.json]


MlflowException: API request to endpoint /api/2.0/mlflow/runs/create failed with error code 401 != 200. Response body: '=============== ATTENTION! ===============

To use authentication, you must first: 
    Get your default access token from: https://dagshub.com/user/settings/tokens
    OR 
    Set a password: https://dagshub.com/user/settings/password 
=========================================='

In [16]:
import os

print("USERNAME:", os.getenv("MLFLOW_TRACKING_USERNAME"))
print("PASSWORD:", os.getenv("MLFLOW_TRACKING_PASSWORD"))


USERNAME: yashviishah
PASSWORD: 3231a18250cb655d859be9f158ddaf2e3716ab82


In [17]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())


Tracking URI: https://dagshub.com/yashviiishah/MLOps.mlflow/


In [18]:
curl -u <your-username>:<your-token> https://dagshub.com/api/v1/repos/<your-username>/<repo-name>


SyntaxError: invalid syntax (781820784.py, line 1)